# Strategies for Blackbox IO Testing

The current (2023-12-01) method is too fickle. We have a separate thread monitoring output on the process and sending STDOUT to a queue which is read by the main thread. But we're getting race conditions on the GIL between the two threads leading to non-deterministic behavior.

Options:
- Python 3.12 `asyncio.subprocess` might work out of the box, but would require that CS 110 use 3.12
  - If we split the blockbox-io functionality out of `byu-pytest-utils`, then perhaps just the CS 235 grader can use it (which happens in docker where we have full control over the python version).
  - In this case, `byu-pytest-utils` needs to provide arms-length support for dialogs
    - `read_dialog(file) -> list[inputs], expected_annotated_output`
    - `score_output(expected_annotated_output, observed_output) -> dict[graded region scores]`
- Send all output to a file and monitor the file for output
  - The 3.12 method may end up just as complicated, so maybe try this first.

## Writing STDOUT to a buffer instead of a pipe

### Can subprocesses write to a BytesIO buffer?

In [1]:
import subprocess, io

In [2]:
buffer = io.BytesIO()
proc = subprocess.Popen('tr a b', stdin=subprocess.PIPE, stdout=buffer, stderr=buffer)

UnsupportedOperation: fileno

**No**. It looks like it needs to be an actual file.

### Using a file handle

In [36]:
import tempfile
import os

In [13]:
with tempfile.TemporaryFile() as output:
    proc = subprocess.Popen('tr a b', shell=True, stdin=subprocess.PIPE, stdout=output, stderr=output)
    proc.stdin.write(b'cat')
    proc.stdin.flush()
    output.flush()
    output.seek(0)
    print(output.read())
    
    proc.stdin.write(b'bat\n')
    output.seek(0)
    print(output.read())
    
    proc.stdin.close()
    proc.wait()
    
    output.seek(0)
    print(output.read())
    

b''
b''
b'cbtbbt\n'


/Users/gbean/opt/miniconda3/envs/py311/lib/python3.11/subprocess.py:1010: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)


### Using PIPE

In [3]:
import os

In [4]:
import time

In [6]:
proc = subprocess.Popen('tr a b', shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
os.set_blocking(proc.stdout.fileno(), False)

for _ in range(5):
    proc.stdin.write(b'cat\n')
    proc.stdin.flush()

    print(proc.stdout.read())

proc.stdin.close()
proc.wait()
print(proc.stdout.read())

None
None
None
None
None
b'cbt\ncbt\ncbt\ncbt\ncbt\n'


In [25]:
%%file pipe_test.py
import os
import subprocess

proc = subprocess.Popen(['python3', '-c', '[print(input("thing: ")) for _ in range(5)]'], 
                        stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
os.set_blocking(proc.stdout.fileno(), False)

for _ in range(5):
    proc.stdin.write(b'cat\n')
    proc.stdin.flush()

    print(proc.stdout.read1())

proc.stdin.close()
proc.wait()
print(proc.stdout.read())

Overwriting pipe_test.py


In [26]:
! python pipe_test.py

b''
b''
b''
b''
b''
b'thing: cat\nthing: cat\nthing: cat\nthing: cat\nthing: cat\n'


## Using `asyncio.subprocess`

In [21]:
%%file async_test.py
import asyncio


async def read(stream):
    buffer = []
    while True:
        try:
            token = await asyncio.wait_for(stream.read(1), 1)
            if not token:
                break
            buffer.append(token.decode())
        except asyncio.TimeoutError:
            break
    
    return ''.join(buffer)
       
    
async def test():
    print('foo')
    proc = await asyncio.subprocess.create_subprocess_shell(
            "python -c '[print(input(\"thing: \")) for _ in range(5)]'", 
        stdin=asyncio.subprocess.PIPE, stdout=asyncio.subprocess.PIPE
        )
    for inp in ['cat', 'bat', 'cow', 'cage', 'foo']:
        print(await read(proc.stdout), end='')
        proc.stdin.write((inp + '\n').encode())
        print(inp)
        await proc.stdin.drain()
    
    print(await read(proc.stdout), end='')
    await proc.wait()

    
asyncio.run(test())

Overwriting async_test.py


In [22]:
!python3 async_test.py

foo
thing: cat
cat
thing: bat
bat
thing: cow
cow
thing: cage
cage
thing: foo
foo
